# 학교 안전사고 현황 분석

- 코드 작성자 : 이유정
- 코드 작성일 : 2024-07-06

맡은 부분 알아서 잘 정리해주세요!!🥰

## 0. 라이브러리 및 데이터 임포트

In [4]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import polars as pl
from datetime import datetime
import pyarrow
import plotly.express as px

In [5]:
df = pd.read_csv('C:/Users/cabby/Desktop/학교안전사고데이터_5개년통합.csv')

df['사고발생일'] = pd.to_datetime(df['사고발생일'])
df['연도'] = df['사고발생일'].map(lambda x : x.year)

df_2019 = df[df['사고발생일'].between('2019-01-01', '2019-12-31')]
df_2020 = df[df['사고발생일'].between('2020-01-01', '2020-12-31')]
df_2021 = df[df['사고발생일'].between('2021-01-01', '2021-12-31')]
df_2022 = df[df['사고발생일'].between('2022-01-01', '2022-12-31')]
df_2023 = df[df['사고발생일'].between('2023-01-01', '2023-12-31')]

## 1. 학교 안전사고 현황 EDA

### 1-1. 연도별 사고 현황

### 1-2. 지역별 사고 현황

- 시도별 사고 현황
    - 시도별 총 사고건수
    - 시도별 연도별 전체 사고건수
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 학교 비율
        -  전체 학교 중 안전사고가 발생한 학교의 비율(2021년 동작구의 전체 학교 N곳 중 M곳의 학교에서 안전사고가 발생했다.)
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 건수
- 시군구별 사고 현황
    - 시군구별 총 사고 건수 (시도별에서 시도를 선택하면 확인할 수 있도록)
    - 시군구별 연도별 총 사고건수
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 학교 비율
        -  전체 학교 중 안전사고가 발생한 학교의 비율(2021년 동작구의 전체 학교 N곳 중 M곳의 학교에서 안전사고가 발생했다.)
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 건수

In [6]:
df_2023.교육청.unique()

'''
교육청(시도)
교육지원청(시군구) --> 시군구별 분석 가능
    교육지원청은 시·도 교육청의 하급 교육행정기관으로, 교육장(3급 또는 4급 상당)의 감독 아래 시·도의 교육·학예에 관한 사무를 일부 분장한다. 
'''

array(['제주시교육지원청', '거창교육지원청', '창원교육지원청', '통영교육지원청', '양산교육지원청', '경상남도교육청',
       '거제교육지원청', '김해교육지원청', '진주교육지원청', '함안교육지원청', '사천교육지원청', '밀양교육지원청',
       '영천교육지원청', '경상북도교육청', '김천교육지원청', '구미교육지원청', '경산교육지원청', '경주교육지원청',
       '포항교육지원청', '성주교육지원청', '익산교육지원청', '전라북도교육청', '전주교육지원청', '김제교육지원청',
       '천안교육지원청', '아산교육지원청', '논산교육지원청', '서천교육지원청', '공주교육지원청', '홍성교육지원청',
       '서산교육지원청', '부여교육지원청', '당진교육지원청', '보은교육지원청', '청주교육지원청', '충청북도교육청',
       '진천교육지원청', '영동교육지원청', '옥천교육지원청', '음성교육지원청', '군포의왕교육지원청', '부천교육지원청',
       '평택교육지원청', '시흥교육지원청', '의정부교육지원청', '수원교육지원청', '고양교육지원청',
       '동두천양주교육지원청', '파주교육지원청', '구리남양주교육지원청', '안산교육지원청', '안양과천교육지원청',
       '화성오산교육지원청', '광명교육지원청', '김포교육지원청', '포천교육지원청', '용인교육지원청', '안성교육지원청',
       '가평교육지원청', '광주하남교육지원청', '세종특별자치시교육청', '강남교육지원청', '울산교육청',
       '강북교육지원청', '서부교육지원청', '동부교육지원청', '남부교육지원청', '북부교육지원청', '인천광역시교육청',
       '해운대교육지원청', '동래교육지원청', '부산광역시교육청', '강동송파교육지원청', '강서양천교육지원청',
       '성동광진교육지원청', '서울특별시교육청', '동작관악교육지원청', '서귀포시교육지원청', '고성교육지원청',
  

### 1-3. 학교급별 사고 현황

- 학교급별 총 사고 수
- 학교급별 학년별 사고 수
- 학교급별 성별별 사고 수
- 학교급별 하루 평균 발생 사고 수
- 학교급별 사고 내용 (사고 형태, 사고 당시 활동, 사고매개물)

##### 1-3-0. 데이터 일부 추출 및 공통 함수 지정

In [56]:
sch_acci_time['사고시간'].nunique()

11

In [54]:
# 연도, 학교급, 사고시간별 사고수 계산
sch_acci_time = sch_df.groupby(['학교급', '사고시간']).size().reset_index(name='사고수')
# 분석 편의를 위해 일부 데이터 값 변경
sch_acci_time = sch_acci_time.replace('점심시간', '식사시간')
sch_acci_time = sch_acci_time.replace('석식시간', '식사시간')
sch_acci_time = sch_acci_time.replace('휴식시간 및 청소시간', '휴식/청소시간')

# 학교급 기준으로 정렬
sch_acci_time['학교급'] = pd.Categorical(sch_acci_time['학교급'], categories=custom_order, ordered=True)
sch_acci_time = sch_acci_time.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
sch_acci_time

,학교급,사고시간,사고수
0,유치원,수업시간,22815
1,유치원,기타,4499
2,유치원,체육수업,3235
3,유치원,식사시간,2578
4,유치원,등하교,2329
...,...,...,...
72,기타학교,쉬는시간,125
73,기타학교,식사시간,74
74,기타학교,기숙사생활,61
75,기타학교,기타,58


In [62]:
sch_acci_time = sch_df.copy()
sch_acci_time = sch_acci_time.replace('점심시간', '식사시간')
sch_acci_time = sch_acci_time.replace('석식시간', '식사시간')
sch_acci_time = sch_acci_time.replace('휴식시간 및 청소시간', '휴식/청소시간')
# 연도, 학교급, 사고시간별 사고수 계산
sch_acci_time = sch_acci_time.groupby(['학교급', '사고시간']).size().reset_index(name='사고수')
# 학교급 기준으로 정렬
sch_acci_time['학교급'] = pd.Categorical(sch_acci_time['학교급'], categories=custom_order, ordered=True)
sch_acci_time = sch_acci_time.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
sch_acci_time

,학교급,사고시간,사고수
0,유치원,수업시간,22815
1,유치원,기타,4499
2,유치원,식사시간,3299
3,유치원,체육수업,3235
4,유치원,등하교,2329
...,...,...,...
60,기타학교,특별활동,151
61,기타학교,체육활동,137
62,기타학교,쉬는시간,125
63,기타학교,기숙사생활,61


In [57]:
sch_df['사고장소'].unique()

array(['부속시설', '통로', '운동장', '교실', '교외활동', '교외'], dtype=object)

In [61]:
# 분석 편의를 위해 일부 데이터 값 변경
sch_acci_place = sch_df.copy()
sch_acci_place = sch_acci_place.replace('교외활동', '교외')
# 연도, 학교급, 사고장소별 사고수 계산
sch_acci_place = sch_acci_place.groupby(['학교급', '사고장소']).size().reset_index(name='사고수')

# 학교급 기준으로 정렬
sch_acci_place['학교급'] = pd.Categorical(sch_acci_place['학교급'], categories=custom_order, ordered=True)
sch_acci_place = sch_acci_place.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
sch_acci_place

,학교급,사고장소,사고수
0,유치원,교실,21144
1,유치원,부속시설,7756
2,유치원,운동장,7330
3,유치원,통로,3565
4,유치원,교외,2259
5,초등학교,부속시설,73355
6,초등학교,운동장,53537
7,초등학교,교실,43159
8,초등학교,통로,40593
9,초등학교,교외,7631


In [68]:
sch_df['사고형태'].unique()

array(['낙상-미끄러짐', '낙상-넘어짐', '기타', '물리적힘 노출', '사람과의 충돌', '낙상-떨어짐', '낙상',
       '염좌·삐임 등 신체 충격'], dtype=object)

In [69]:
# 분석 편의를 위해 일부 데이터 값 변경
sch_acci_type = sch_df.copy()
sch_acci_type = sch_acci_type.replace('낙상-미끄러짐', '낙상')
sch_acci_type = sch_acci_type.replace('낙상-넘어짐', '낙상')
sch_acci_type = sch_acci_type.replace('낙상-떨어짐', '낙상')
sch_acci_type = sch_acci_type.replace('염좌·삐임 등 신체 충격', '염좌/삐임/신체 충격')
# 연도, 학교급, 사고형태별 사고수 계산
sch_acci_type = sch_acci_type.groupby(['학교급', '사고형태']).size().reset_index(name='사고수')
# 학교급 기준으로 정렬
sch_acci_type['학교급'] = pd.Categorical(sch_acci_type['학교급'], categories=custom_order, ordered=True)
sch_acci_type = sch_acci_type.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
sch_acci_type

,학교급,사고형태,사고수
0,유치원,물리적힘 노출,18861
1,유치원,낙상,13476
2,유치원,기타,6315
3,유치원,사람과의 충돌,2759
4,유치원,염좌/삐임/신체 충격,643
5,초등학교,낙상,86300
6,초등학교,물리적힘 노출,85969
7,초등학교,사람과의 충돌,19237
8,초등학교,염좌/삐임/신체 충격,13703
9,초등학교,기타,13066


In [18]:
# 연도, 학교급, 사고형태별 사고수 계산
sch_acci_type = sch_df.groupby(['학교급', '사고형태']).size().reset_index(name='사고수')

# 학교급 기준으로 정렬
sch_acci_type['학교급'] = pd.Categorical(sch_acci_type['학교급'], categories=custom_order, ordered=True)
sch_acci_type = sch_acci_type.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
sch_acci_type

,학교급,사고형태,사고수
0,유치원,물리적힘 노출,18861
1,유치원,낙상-넘어짐,7472
2,유치원,기타,6315
3,유치원,사람과의 충돌,2759
4,유치원,낙상-미끄러짐,2692
5,유치원,낙상,2114
6,유치원,낙상-떨어짐,1198
7,유치원,염좌·삐임 등 신체 충격,643
8,초등학교,물리적힘 노출,85969
9,초등학교,낙상-넘어짐,43106


In [3]:
# 기본 정보 확인
print(df.columns)
print(df.학교급.unique())
df.groupby('학교급').count()['구분']

Index(['구분', '학교급', '지역', '교육청', '설립유형', '사고자구분', '사고자성별', '사고자학년', '사고발생일',
       '사고발생요일', '사고발생시각', '사고시간', '사고장소', '사고부위', '사고형태', '사고당시활동', '사고매개물',
       '연도'],
      dtype='object')
['중학교' '초등학교' '고등학교' '유치원' '특수학교' '기타학교']


학교급
고등학교    130050
기타학교      1954
유치원      42403
중학교     216165
초등학교    218830
특수학교      2558
Name: 구분, dtype: int64

In [6]:
# 사고자구분 중 '일반학생', '특수학교(학급)학생', '체육특기학생' 값만 추출
print('추출 전:', df.사고자구분.unique())

sch_df = df[df['사고자구분'].isin(['일반학생', '특수학교(학급)학생', '체육특기학생'])]
print('추출 후:', sch_df.사고자구분.unique())

추출 전: ['일반학생' '특수학교(학급)학생' '체육특기학생' '교직원' '교육활동참여자' '교직원(계약직)' '교직원(일반직)' nan
 '교직원(교원)' '일반인' '기타(일반인 등)']
추출 후: ['일반학생' '특수학교(학급)학생' '체육특기학생']


In [6]:
# 그래프 출력 함수_piechart 

def create_piechart(df, year_col, name_col, value_col, name_order, colors):
    
    # 연도 목록 생성
    years = df[year_col].unique()

    # 그래프 생성
    fig = px.pie(df[df[year_col] == years[0]], 
                 names=name_col, 
                 values=value_col,
                 category_orders={name_col: name_order},
                 color=name_col,
                 color_discrete_map=colors,
                 title=f"{years[0]}년 {name_col}별 {value_col}")

    # 드롭다운 메뉴 생성
    dropdown_buttons = [
        {
            'label': str(year),
            'method': 'update',
            'args': [
                {'values': [df[df[year_col] == year][value_col]],
                 'labels': [df[df[year_col] == year][name_col]]}, 
                {'marker.colors': [colors[label] for label in name_order if label in df[df[year_col] == year][name_col].values],
                 'title': f"{year}년 {name_col}별 {value_col}"}
            ]
        }
        for year in years
    ]
     
    # 레이아웃 설정
    fig.update_layout(
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True
        }],
        font=dict({'family':'KoPubWorld돋움체_Pro', 'color':'black'}),
        hoverlabel=dict(font_size=15,
                        font_family="KoPubWorld돋움체_Pro"),
        paper_bgcolor='white',  
        plot_bgcolor='white',    
        legend=dict(traceorder='normal')
    )
    
    # 퍼센트 포맷 및 hover 설정
    fig.update_traces(texttemplate='%{percent:.2%}',
                      hovertemplate='<b>%{label}</b><br>사고수: %{value}<br>')

    # 그래프 출력
    fig.show()

In [10]:
# 그래프 출력 함수_성별별 사고 수

# 그래프 색 지정
colors = {
    '남': '#89c2d9',
    '여': '#ffbf69'
}

def create_chart_gender(df, x, y, color, sch_type):
    
    # 그래프 생성
    fig = px.bar(df, x=x, y=y, color=color, color_discrete_map=colors, 
                 title=f"{sch_type} {color}별 {y}", custom_data=['사고자성별','사고수_비율'])
    
    # y축 형식 설정
    fig.update_layout(yaxis=dict(tickformat=","),
                      font=dict({'family':'KoPubWorld돋움체_Pro', 'color':'black'}),
                      hoverlabel=dict(font_size=15,
                                      font_family="KoPubWorld돋움체_Pro"),
                      paper_bgcolor='white', 
                      plot_bgcolor='white'
                      )

    # 텍스트 설정
    fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>%{x}<br>사고수: %{y:,}<br>사고수 비율: %{customdata[1]}%<extra></extra>')

    # 그래프 출력
    fig.show()

In [42]:
# 그래프 출력 함수_barplot

# 그래프 색 지정
colors = {
    '2019': '#a9d6e5',
    '2020': '#468faf',
    '2021': '#2a6f97',
    '2022': '#014f86',
    '2023': '#013a63'
}

def create_chart(df, x, y, color, sch_type):

    # 그래프 생성
    fig = px.bar(df, x=x, y=y, color=color, color_discrete_map=colors,
                 barmode='group', title=f"{sch_type} {x}별 {y}", custom_data=[color])

    # 레이아웃 설정
    fig.update_layout(bargap=0.15, height=450, width=950,
                      xaxis={'visible': True, 'showticklabels': True, 'title': x},
                      yaxis={'visible': False, 'showticklabels': False},
                      font=dict({'family':'KoPubWorld돋움체_Pro', 'color':'black'}),
                      hoverlabel=dict(font_size=15,
                                      font_family="KoPubWorld돋움체_Pro"),
                      paper_bgcolor='white', 
                      plot_bgcolor='white',
                      showlegend=True, margin=dict(l=2, r=2),
                      legend=dict(orientation="v", yanchor= "top", y=1.1, xanchor= "right", x=1, font=dict(size=10)))

    # 텍스트 설정
    fig.update_traces(texttemplate=None, hovertemplate='<b>%{customdata[0]}</b><br>%{x}<br>사고수: %{y}<extra></extra>')

    # 그래프 출력
    fig.show()

#### 1-3-1. 학교급별 총 사고 수

In [7]:
# 연도, 학교급 기준으로 groupby
sch_tot_acci = sch_df.groupby(['연도', '학교급']).size().reset_index(name='총 사고수')

# 연도 type str로 변경
sch_tot_acci['연도'] = sch_tot_acci['연도'].astype(str)

# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_tot_acci['학교급'] = pd.Categorical(sch_tot_acci['학교급'], categories=custom_order, ordered=True)
sch_tot_acci = sch_tot_acci.sort_values(['연도', '학교급']).reset_index(drop=True)

sch_tot_acci

,연도,학교급,총 사고수
0,2019,유치원,8735
1,2019,초등학교,53728
2,2019,중학교,43579
3,2019,고등학교,31601
4,2019,특수학교,545
5,2019,기타학교,357
6,2020,유치원,6026
7,2020,초등학교,10928
8,2020,중학교,11214
9,2020,고등학교,10752


In [9]:
# 학교급 순서 지정
sch_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]

# 그래프 색 지정
colors = {
    '유치원': '#a9d6e5',
    '초등학교': '#89c2d9',
    '중학교': '#468faf',
    '고등학교': '#2a6f97',
    '특수학교': '#014f86',
    '기타학교': '#013a63'
}

# 그래프 출력 함수 실행
create_piechart(sch_tot_acci, '연도', '학교급', '총 사고수', sch_order, colors)

In [15]:
sch_df[sch_df['학교급'] == '초등학교']['사고자학년'].unique()

array(['2학년', '5학년', '6학년', '3학년', '1학년', '4학년', 'nan', '유아'],
      dtype=object)

In [26]:
sch_df_level_div = sch_df.copy()
sch_df_level_div['학교급'] = sch_df_level_div.apply(update_school_level, axis=1)

np.int64(0)

In [28]:
# 초등학교 저학년과 고학년으로 구분하는 함수
def update_school_level(row):
    if row['학교급'] == '초등학교':
        if row['사고자학년'] == '1학년' or row['사고자학년'] == '2학년' or row['사고자학년'] == '3학년':
            return '초등학교_저학년'
        elif row['사고자학년'] == '4학년' or row['사고자학년'] == '5학년' or row['사고자학년'] == '6학년':
            return '초등학교_고학년'
    return row['학교급']

def create_piechart(df, year, name_col, value_col, name_order, colors):
    year_df = df[df['연도'] == year]
    fig = px.pie(year_df,
                 names=name_col,
                 values=value_col,
                 category_orders={name_col: name_order},
                 color=name_col,
                 color_discrete_map=colors,
                 title=f"{year}년 {name_col}별 {value_col}")
    fig.update_traces(texttemplate='%{percent:.2%}',
                      hovertemplate='<b>%{label}</b><br>사고수: %{value}<br>')
    return fig

# 초등학교 저학년과 고학년으로 구분한 데이터프레임
sch_df_level_div = sch_df.copy()
sch_df_level_div['학교급'] = sch_df_level_div.apply(update_school_level, axis=1)

# 연도, 학교급 기준으로 groupby
sch_tot_acci = sch_df_level_div.groupby(['연도', '학교급']).size().reset_index(name='총 사고수')
# 연도 type str로 변경
sch_tot_acci['연도'] = sch_tot_acci['연도'].astype(str)
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교_저학년", "초등학교_고학년", "중학교", "고등학교", "특수학교", "기타학교"]
sch_tot_acci['학교급'] = pd.Categorical(sch_tot_acci['학교급'], categories=custom_order, ordered=True)
sch_tot_acci = sch_tot_acci.sort_values(['연도', '학교급']).reset_index(drop=True)
# update_school_level 함수 적용 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_tot_acci = sch_tot_acci.dropna()
# 최종 결과 확인
sch_tot_acci

,연도,학교급,총 사고수
0,2019,유치원,8735
1,2019,초등학교_저학년,21340
2,2019,초등학교_고학년,32195
3,2019,중학교,43579
4,2019,고등학교,31601
5,2019,특수학교,545
6,2019,기타학교,357
8,2020,유치원,6026
9,2020,초등학교_저학년,4674
10,2020,초등학교_고학년,6242


In [34]:
custom_order = ["유치원", "초등학교_저학년", "초등학교_고학년", "중학교", "고등학교", "특수학교", "기타학교"]
colors = {"유치원": '#5c7dd2', 
          "초등학교_저학년": '#92b8ff', 
          "초등학교_고학년": '#aeceff', 
          "중학교": '#c7e4ff', 
          "고등학교": '#c3b7eb', 
          "특수학교": '#9590e6', 
          "기타학교": '#837ed5'}
st.plotly_chart(create_piechart(sch_tot_acci, 2019, '학교급', '총 사고수', custom_order, colors))

DeltaGenerator()

In [39]:
sch_df_year = sch_df[sch_df['연도'] == 2019]
sch_df_year

,구분,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생일,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물,연도
0,A0000209,중학교,경기,안양과천교육지원청,공립,일반학생,남,1학년,2019-01-02,수,09:00,수업시간,부속시설,다리,낙상-미끄러짐,기타운동,건물(문/창문/바닥/벽 등),2019
1,A0000261,초등학교,경기,안양과천교육지원청,공립,일반학생,남,2학년,2019-01-02,수,08:50,등하교,통로,치아(구강),낙상-넘어짐,보행/주행,건물(문/창문/바닥/벽 등),2019
2,A0000262,중학교,경기,부천교육지원청,공립,일반학생,여,1학년,2019-01-02,수,10:20,체육수업,부속시설,손,기타,구기운동,기타,2019
3,A0000310,초등학교,광주,서부교육지원청,공립,일반학생,여,2학년,2019-01-02,수,10:35,휴식시간 및 청소시간,운동장,머리(두부),낙상-넘어짐,장난/놀이,자연(사람/동물/식물 등),2019
4,A0000316,중학교,충북,청주교육지원청,공립,일반학생,남,2학년,2019-01-02,수,09:10,체육수업,부속시설,발,낙상-미끄러짐,보행/주행,건물(문/창문/바닥/벽 등),2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411839,D0142392,초등학교,서울,강서양천교육지원청,공립,일반학생,남,6학년,2019-12-20,금,10:30,학교행사,교외활동,팔,물리적힘 노출,기타운동,기타,2019
417111,D0147665,초등학교,경기,군포의왕교육지원청,공립,일반학생,남,3학년,2019-12-31,화,09:45,휴식시간 및 청소시간,교실,치아(구강),낙상-넘어짐,식사/수면/휴식,건물(문/창문/바닥/벽 등),2019
425378,E0006593,고등학교,광주,광주광역시교육청,사립,일반학생,남,1학년,2019-06-25,화,10:00,학교행사,운동장,다리,낙상,구기운동,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),2019
458885,E0040101,고등학교,충남,논산교육지원청,공립,체육특기학생,남,3학년,2019-02-20,수,14:00,특별활동,교외,다리,기타,기타운동,기타,2019


In [36]:
sch_df_year = sch_df[sch_df['연도'] == '2019']
# 학교급별 사고수 groupby
sch_tot_acci = sch_df_year.groupby(['학교급']).size().reset_index(name='총 사고수')
# 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_tot_acci['학교급'] = pd.Categorical(sch_tot_acci['학교급'], categories=custom_order, ordered=True)
sch_tot_acci = sch_tot_acci.sort_values(['총 사고수', '학교급'], ascending=[False, True]).reset_index(drop=True)
# 최대값 추출 및 정수 변환
#max_value = int(sch_tot_acci["총 사고수"].max())
sch_tot_acci
#print(max_value)

,학교급,총 사고수


#### 1-3-2. 학교급별 학년별 사고 수

In [78]:
# sch_grade_acci 데이터프레임
sch_grade_acci = sch_df.groupby(['연도', '학교급', '사고자학년']).size().reset_index(name='사고수')
# 유치원, 기타학교 값 제외
sch_grade_acci = sch_grade_acci[(sch_grade_acci['학교급']!='기타학교') & (sch_grade_acci['학교급'] != '유치원')]
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_grade_acci['학교급'] = pd.Categorical(sch_grade_acci['학교급'], categories=custom_order, ordered=True)
sch_grade_acci = sch_grade_acci.sort_values(['학교급','연도']).reset_index(drop=True)
sch_grade_acci[sch_grade_acci['학교급'] == '특수학교']['사고자학년'].unique()

array(['1학년', '2학년', '3학년', '4학년', '5학년', '6학년', 'nan', '유아'],
      dtype=object)

In [120]:
# sch_tot_acci2 데이터프레임
sch_tot_acci2 = sch_df.copy()
# 초등학교 저학년/고학년 구분
sch_tot_acci2.loc[(sch_tot_acci2['학교급']=='초등학교')&(sch_tot_acci2['사고자학년'].isin(['1학년','2학년','3학년'])),'학교급']='초등학교(저학년)'
sch_tot_acci2.loc[(sch_tot_acci2['학교급']=='초등학교')&(sch_tot_acci2['사고자학년'].isin(['4학년','5학년','6학년'])),'학교급']='초등학교(고학년)'
sch_tot_acci2['학교급'].unique()

array(['중학교', '초등학교(저학년)', '초등학교(고학년)', '고등학교', '유치원', '초등학교', '특수학교',
       '기타학교'], dtype=object)

In [107]:
 # sch_grade_acci 데이터프레임
# 유치원, 기타학교 값 제외
sch_grade_acci = sch_df[(sch_df['학교급']!='기타학교') & (sch_df['학교급'] != '유치원')]
# 연도, 학교급, 사고자학년 기준으로 groupby
sch_grade_acci = sch_grade_acci.groupby(['연도', '학교급', '사고자학년']).size().reset_index(name='사고수')
sch_grade_acci.isna().sum()

연도       0
학교급      0
사고자학년    0
사고수      0
dtype: int64

In [115]:
sch_df['사고자학년'].unique()

array(['1학년', '2학년', '3학년', '5학년', '6학년', '유아', '4학년', 'nan'],
      dtype=object)

In [117]:
# sch_grade_acci 데이터프레임
# '학교급'의 '유치원', '기타학교' 값 '사고자학년'의 'nan' 값제외
sch_grade_acci = sch_df[
            (sch_df['학교급'] != '기타학교') &
            (sch_df['학교급'] != '유치원') &
            (sch_df['사고자학년'] != 'nan')
            ]

# 연도, 학교급, 사고자학년 기준으로 groupby
sch_grade_acci = sch_grade_acci.groupby(['연도', '학교급', '사고자학년']).size().reset_index(name='사고수')

# '초등학교'의 '사고자학년'에서 '유아' 값을 제외
sch_grade_acci = sch_grade_acci[~((sch_grade_acci['학교급'] == '초등학교') & 
                                          (sch_grade_acci['사고자학년'] == '유아'))]
# '중학교'의 '사고자학년'에서 '4학년'과 '5학년' 값을 제외
sch_grade_acci = sch_grade_acci[~((sch_grade_acci['학교급'] == '중학교') & 
                                          ((sch_grade_acci['사고자학년'] == '4학년') | 
                                           (sch_grade_acci['사고자학년'] == '5학년')))]

sch_grade_acci['사고수 비율'] = sch_grade_acci.groupby(['연도', '학교급'])['사고수'].transform(lambda x: (x / x.sum() * 100).round(2))

# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교"]
sch_grade_acci['학교급'] = pd.Categorical(sch_grade_acci['학교급'], categories=custom_order, ordered=True)
sch_grade_acci = sch_grade_acci.sort_values(['학교급','연도']).reset_index(drop=True)

sch_grade_acci

,연도,학교급,사고자학년,사고수,사고수 비율
0,2019,초등학교,1학년,7014,13.10
1,2019,초등학교,2학년,6578,12.29
2,2019,초등학교,3학년,7748,14.47
3,2019,초등학교,4학년,8669,16.19
4,2019,초등학교,5학년,10837,20.24
5,2019,초등학교,6학년,12689,23.70
6,2020,초등학교,1학년,1670,15.30
7,2020,초등학교,2학년,1514,13.87
8,2020,초등학교,3학년,1490,13.65
9,2020,초등학교,4학년,1839,16.85


##### (1) 초등학교

In [109]:
# 학교급 중 초등학교 추출
sch_es = sch_df[sch_df['학교급'] == '초등학교']
print(sch_es['사고자학년'].unique())

# '사고자학년' 컬럼에서 각 값의 행 수 계산
print(sch_es['사고자학년'].value_counts())

# '사고자학년' 중 유아 값 제외
sch_es = sch_es[sch_es['사고자학년'] != '유아']

['2학년' '5학년' '6학년' '3학년' '1학년' '4학년' 'nan' '유아']
사고자학년
6학년    54046
5학년    47562
4학년    37448
3학년    29711
2학년    24676
1학년    24623
nan      193
유아        16
Name: count, dtype: int64


In [111]:
# 사고발생일 중 연도만 추출 후 연도, 사고자학년 기준으로 groupby
sch_es_grade = sch_es.groupby(['연도', '사고자학년']).size().reset_index(name='사고수')

# 연도, 사고자학년 기준으로 정렬
sch_es_grade = sch_es_grade.sort_values(['연도', '사고자학년']).reset_index(drop=True)

# 그래프 해석 편의를 위해 컬럼명 변경
sch_es_grade.rename(columns={'사고자학년': '초등학교_사고자학년'}, inplace=True)

sch_es_grade

,연도,초등학교_사고자학년,사고수
0,2019,1학년,7014
1,2019,2학년,6578
2,2019,3학년,7748
3,2019,4학년,8669
4,2019,5학년,10837
5,2019,6학년,12689
6,2019,nan,192
7,2020,1학년,1670
8,2020,2학년,1514
9,2020,3학년,1490


In [39]:
# 사고발생일 중 연도만 추출 후 연도, 사고자학년 기준으로 groupby
sch_es_grade = sch_es.groupby(['연도', '사고자학년']).size().reset_index(name='사고수')

# 연도 type str로 변경
sch_es_grade['연도'] = sch_es_grade['연도'].astype(str)

# 연도, 사고자학년 기준으로 정렬
sch_es_grade = sch_es_grade.sort_values(['연도', '사고자학년']).reset_index(drop=True)

# 그래프 해석 편의를 위해 컬럼명 변경
sch_es_grade.rename(columns={'사고자학년': '초등학교_사고자학년'}, inplace=True)

sch_es_grade

,연도,초등학교_사고자학년,사고수
0,2019,1학년,7014
1,2019,2학년,6578
2,2019,3학년,7748
3,2019,4학년,8669
4,2019,5학년,10837
5,2019,6학년,12689
6,2020,1학년,1670
7,2020,2학년,1514
8,2020,3학년,1490
9,2020,4학년,1839


In [40]:
# 사고자학년 순서 지정
es_order = ["1학년", "2학년", "3학년", "4학년", "5학년", "6학년"]

# 그래프 색 지정
colors = {
    '1학년': '#a9d6e5',
    '2학년': '#89c2d9',
    '3학년': '#468faf',
    '4학년': '#2a6f97',
    '5학년': '#014f86',
    '6학년': '#013a63'
}

# 그래프 출력 함수 실행
create_piechart(sch_es_grade, '연도', '초등학교_사고자학년', '사고수', es_order, colors)

##### (2) 중학교

In [35]:
# 학교급 중 중학교 추출
sch_ms = df[df['학교급'] == '중학교']
print(sch_ms['사고자학년'].unique())

# '사고자학년' 컬럼에서 각 값의 행 수 계산
print(sch_ms['사고자학년'].value_counts())

# '사고자학년' 중 4학년, 5학년 값 제외
temp = ['4학년', '5학년']
sch_ms = sch_ms[~sch_ms['사고자학년'].isin(temp)]

['1학년' '2학년' '3학년' nan '5학년' '4학년']
사고자학년
1학년    76788
2학년    70023
3학년    69111
4학년        2
5학년        1
Name: count, dtype: int64


In [36]:
# 사고발생일 중 연도만 추출 후 연도, 사고자학년 기준으로 groupby
sch_ms_grade = sch_ms.groupby(['연도', '사고자학년']).size().reset_index(name='사고수')

# 연도 type str로 변경
sch_ms_grade['연도'] = sch_ms_grade['연도'].astype(str)

# 연도, 사고자학년 기준으로 정렬
sch_ms_grade = sch_ms_grade.sort_values(['연도', '사고자학년']).reset_index(drop=True)

# 그래프 해석 편의를 위해 컬럼명 변경
sch_ms_grade.rename(columns={'사고자학년': '중학교_사고자학년'}, inplace=True)

sch_ms_grade

,연도,중학교_사고자학년,사고수
0,2019,1학년,15346
1,2019,2학년,13882
2,2019,3학년,14365
3,2020,1학년,4275
4,2020,2학년,3486
5,2020,3학년,3455
6,2021,1학년,12162
7,2021,2학년,11678
8,2021,3학년,10199
9,2022,1학년,19672


In [37]:
# 사고자학년 순서 지정
ms_order = ["1학년", "2학년", "3학년"]

# 그래프 색 지정
colors = {
    '1학년': '#a9d6e5',
    '2학년': '#89c2d9',
    '3학년': '#468faf',
}

# 그래프 출력 함수 실행
create_piechart(sch_ms_grade, '연도', '중학교_사고자학년', '사고수', ms_order, colors)

##### (3) 고등학교

In [32]:
# 학교급 중 고등학교 추출
sch_hs = sch_df[sch_df['학교급'] == '고등학교']
print(sch_hs['사고자학년'].unique())

['1학년' '2학년' '3학년' nan]


In [33]:
# 사고발생일 중 연도만 추출 후 연도, 사고자학년 기준으로 groupby
sch_hs_grade = sch_hs.groupby(['연도', '사고자학년']).size().reset_index(name='사고수')

# 연도 type str로 변경
sch_hs_grade['연도'] = sch_hs_grade['연도'].astype(str)

# 연도, 사고자학년 기준으로 정렬
sch_hs_grade = sch_hs_grade.sort_values(['연도', '사고자학년']).reset_index(drop=True)

# 그래프 해석 편의를 위해 컬럼명 변경
sch_hs_grade.rename(columns={'사고자학년': '고등학교_사고자학년'}, inplace=True)

sch_hs_grade

,연도,고등학교_사고자학년,사고수
0,2019,1학년,11630
1,2019,2학년,11889
2,2019,3학년,8080
3,2020,1학년,3727
4,2020,2학년,3958
5,2020,3학년,3067
6,2021,1학년,6805
7,2021,2학년,6963
8,2021,3학년,6054
9,2022,1학년,11547


In [34]:
# 사고자학년 순서 지정
hs_order = ["1학년", "2학년", "3학년"]

# 그래프 색 지정
colors = {
    '1학년': '#a9d6e5',
    '2학년': '#89c2d9',
    '3학년': '#468faf',
}

# 그래프 출력 함수 실행
create_piechart(sch_hs_grade, '연도', '고등학교_사고자학년', '사고수', hs_order, colors)

##### (4) 특수학교

In [28]:
# 학교급 중 특수학교 추출
sch_ss = sch_df[sch_df['학교급'] == '특수학교']
print(sch_ss['사고자학년'].unique())

# '사고자학년' 컬럼에서 각 값의 행 수 계산
print(sch_ss['사고자학년'].value_counts())

['3학년' '4학년' '5학년' '1학년' '2학년' '6학년' nan '유아']
사고자학년
1학년    764
2학년    716
3학년    479
5학년    146
6학년    145
4학년    128
유아      73
Name: count, dtype: int64


In [29]:
# 사고발생일 중 연도만 추출 후 연도, 사고자학년 기준으로 groupby
sch_ss_grade = sch_ss.groupby(['연도', '사고자학년']).size().reset_index(name='사고수')

# 연도 type str로 변경
sch_ss_grade['연도'] = sch_ss_grade['연도'].astype(str)

# 정렬 순서 지정
ss_order = ["유아", "1학년", "2학년", "3학년", "4학년", "5학년", "6학년"]
cat_type = CategoricalDtype(categories=ss_order, ordered=True)
sch_ss_grade['사고자학년'] = sch_ss_grade['사고자학년'].astype(cat_type)
 
# 연도, 사고자학년 기준으로 정렬
sch_ss_grade = sch_ss_grade.sort_values(by=['연도', '사고자학년'])

# 그래프 해석 편의를 위해 컬럼명 변경
sch_ss_grade.rename(columns={'사고자학년': '특수학교_사고자학년'}, inplace=True)

sch_ss_grade

,연도,특수학교_사고자학년,사고수
0,2019,1학년,166
1,2019,2학년,171
2,2019,3학년,100
3,2019,4학년,22
4,2019,5학년,35
5,2019,6학년,31
12,2020,유아,11
6,2020,1학년,87
7,2020,2학년,61
8,2020,3학년,62


In [30]:
# 그래프 해석 편의를 위해 모든 학년을 포함
temp_years = sch_ss_grade['연도'].unique()
temp_grades = ss_order
temp_data = pd.MultiIndex.from_product([temp_years, temp_grades], names=['연도', '특수학교_사고자학년']).to_frame(index=False)
sch_ss_grade = pd.merge(temp_data, sch_ss_grade, on=['연도', '특수학교_사고자학년'], how='left').fillna(0)
sch_ss_grade

,연도,특수학교_사고자학년,사고수
0,2019,유아,0.0
1,2019,1학년,166.0
2,2019,2학년,171.0
3,2019,3학년,100.0
4,2019,4학년,22.0
5,2019,5학년,35.0
6,2019,6학년,31.0
7,2020,유아,11.0
8,2020,1학년,87.0
9,2020,2학년,61.0


In [31]:
# 그래프 색 지정
colors = {
    '유아': '#a9d6e5',
    '1학년': '#89c2d9',
    '2학년': '#468faf',
    '3학년': '#2a6f97',
    '4학년': '#014f86',
    '5학년': '#013a63',
    '6학년': '#012a4a'
}

# 그래프 출력 함수 실행
create_piechart(sch_ss_grade, '연도', '특수학교_사고자학년', '사고수', ss_order, colors)

#### 1-3-3. 학교급별 성별별 사고 수

In [12]:
# 사고발생일 중 연도만 추출 후 연도, 학교급, 사고자성별 기준으로 groupby
sch_gender_acci = sch_df.groupby(['연도', '학교급', '사고자성별']).size().reset_index(name='사고수')

# 연도 type str로 변경
sch_gender_acci['연도'] = sch_gender_acci['연도'].astype(str)

# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_gender_acci['학교급'] = pd.Categorical(sch_gender_acci['학교급'], categories=custom_order, ordered=True)
sch_gender_acci = sch_gender_acci.sort_values(['학교급','연도']).reset_index(drop=True)

# 사고자성별별 사고수 비율 계산
sch_gender_acci['사고수_비율'] = sch_gender_acci.groupby(['연도', '학교급'])['사고수'].transform(lambda x: (x / x.sum() * 100).round(2))

sch_gender_acci

C:\Users\cabby\AppData\Local\Temp\ipykernel_30920\3286144405.py:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,연도,학교급,사고자성별,사고수,사고수_비율
0,2019,유치원,남,5269,60.32
1,2019,유치원,여,3466,39.68
2,2020,유치원,남,3600,59.74
3,2020,유치원,여,2426,40.26
4,2021,유치원,남,5557,60.49
5,2021,유치원,여,3630,39.51
6,2022,유치원,남,5486,60.26
7,2022,유치원,여,3618,39.74
8,2023,유치원,남,5312,59.01
9,2023,유치원,여,3690,40.99


In [13]:
# 그래프 출력 함수 실행
create_chart_gender(sch_gender_acci[sch_gender_acci['학교급']=='유치원'], '연도', '사고수', '사고자성별', '유치원')

In [14]:
# 그래프 출력 함수 실행
create_chart_gender(sch_gender_acci[sch_gender_acci['학교급']=='초등학교'], '연도', '사고수', '사고자성별', '초등학교')

In [15]:
# 그래프 출력 함수 실행
create_chart_gender(sch_gender_acci[sch_gender_acci['학교급']=='중학교'], '연도', '사고수', '사고자성별', '중학교')

In [16]:
# 그래프 출력 함수 실행
create_chart_gender(sch_gender_acci[sch_gender_acci['학교급']=='고등학교'], '연도', '사고수', '사고자성별', '고등학교')

In [17]:
# 그래프 출력 함수 실행
create_chart_gender(sch_gender_acci[sch_gender_acci['학교급']=='특수학교'], '연도', '사고수', '사고자성별', '특수학교')

In [18]:
# 그래프 출력 함수 실행
create_chart_gender(sch_gender_acci[sch_gender_acci['학교급']=='기타학교'], '연도', '사고수', '사고자성별', '기타학교')

#### 1-3-4.학교급별 하루 평균 발생 사고 수

In [12]:
# 연도, 학교급 기준으로 groupby 후 각 연도와 학교급별 총 사고발생일 수 계산
sch_aver_acci = sch_df.groupby(['연도', '학교급']).agg(총사고수=('사고발생일', 'count')).reset_index()

# 연도 type str로 변경
sch_aver_acci['연도'] = sch_aver_acci['연도'].astype(str)

# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_aver_acci['학교급'] = pd.Categorical(sch_aver_acci['학교급'], categories=custom_order, ordered=True)
sch_aver_acci = sch_aver_acci.sort_values(['연도', '학교급']).reset_index(drop=True)

# 하루 평균 사고 발생 수 계산
sch_aver_acci['하루평균사고수'] = round(sch_aver_acci['총사고수'] / 365, 2)

sch_aver_acci 

,연도,학교급,총사고수,하루평균사고수
0,2019,유치원,8735,23.93
1,2019,초등학교,53728,147.20
2,2019,중학교,43579,119.39
3,2019,고등학교,31601,86.58
4,2019,특수학교,545,1.49
5,2019,기타학교,357,0.98
6,2020,유치원,6026,16.51
7,2020,초등학교,10928,29.94
8,2020,중학교,11214,30.72
9,2020,고등학교,10752,29.46


In [13]:
# 전년 대비 증감률 계산
sch_aver_acci['전년대비증감률'] = sch_aver_acci.groupby('학교급')['하루평균사고수'].pct_change().fillna(0) * 100
sch_aver_acci

C:\Users\cabby\AppData\Local\Temp\ipykernel_19628\36661600.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sch_aver_acci['전년대비증감률'] = sch_aver_acci.groupby('학교급')['하루평균사고수'].pct_change().fillna(0) * 100


,연도,학교급,총사고수,하루평균사고수,전년대비증감률
0,2019,유치원,8735,23.93,0.000000
1,2019,초등학교,53728,147.20,0.000000
2,2019,중학교,43579,119.39,0.000000
3,2019,고등학교,31601,86.58,0.000000
4,2019,특수학교,545,1.49,0.000000
5,2019,기타학교,357,0.98,0.000000
6,2020,유치원,6026,16.51,-31.007104
7,2020,초등학교,10928,29.94,-79.660326
8,2020,중학교,11214,30.72,-74.269202
9,2020,고등학교,10752,29.46,-65.973666


#### 1-3-5. 학교급별 사고 내용

In [46]:
sch_df.columns

Index(['구분', '학교급', '지역', '교육청', '설립유형', '사고자구분', '사고자성별', '사고자학년', '사고발생일',
       '사고발생요일', '사고발생시각', '사고시간', '사고장소', '사고부위', '사고형태', '사고당시활동', '사고매개물',
       '연도'],
      dtype='object')

In [47]:
sch_df['사고시간'].unique()

array(['수업시간', '등하교', '체육수업', '휴식시간 및 청소시간', '점심시간', '특별활동', '학교행사',
       '석식시간', '기숙사생활', '기타', '식사시간', '체육활동', '쉬는시간'], dtype=object)

##### (1)  사고형태

In [5]:
# 연도, 학교급, 사고형태별 사고수 계산
sch_acci_type = sch_df.groupby(['연도', '학교급', '사고형태']).size().reset_index(name='사고수')

# 연도 type str로 변경
sch_acci_type['연도'] = sch_acci_type['연도'].astype(str)

# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_acci_type['학교급'] = pd.Categorical(sch_acci_type['학교급'], categories=custom_order, ordered=True)
sch_acci_type = sch_acci_type.sort_values(['연도', '학교급']).reset_index(drop=True)

sch_acci_type

,연도,학교급,사고형태,사고수
0,2019,유치원,기타,1229
1,2019,유치원,낙상-넘어짐,2005
2,2019,유치원,낙상-떨어짐,318
3,2019,유치원,낙상-미끄러짐,444
4,2019,유치원,물리적힘 노출,3703
...,...,...,...,...
184,2023,기타학교,기타,31
185,2023,기타학교,낙상,196
186,2023,기타학교,물리적힘 노출,175
187,2023,기타학교,사람과의 충돌,61


In [63]:
sch_df['사고부위'].unique()

array(['다리', '치아(구강)', '손', '머리(두부)', '발', '기타', '팔', '흉복부', '복합부위'],
      dtype=object)

In [67]:
sch_acci_part = sch_df.copy()
sch_acci_part['사고부위'] = sch_acci_part['사고부위'].str.replace(r'\([^)]*\)', '', regex=True).str.strip()
# 연도, 학교급, 사고부위별 사고수 계산
sch_acci_part = sch_acci_part.groupby(['학교급', '사고부위']).size().reset_index(name='사고수')
# 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_acci_part['학교급'] = pd.Categorical(sch_acci_part['학교급'], categories=custom_order, ordered=True)
sch_acci_part = sch_acci_part.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
sch_acci_part

,학교급,사고부위,사고수
0,유치원,머리,16983
1,유치원,손,5440
2,유치원,치아,5026
3,유치원,발,4502
4,유치원,팔,3313
5,유치원,복합부위,3164
6,유치원,기타,2259
7,유치원,다리,1064
8,유치원,흉복부,303
9,초등학교,손,56248


In [7]:
# 사고부위 5개년 누적
# 연도, 학교급, 사고부위별 사고수 계산
sch_acci_part = sch_df.groupby(['학교급', '사고부위']).size().reset_index(name='사고수')

# 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_acci_part['학교급'] = pd.Categorical(sch_acci_part['학교급'], categories=custom_order, ordered=True)
sch_acci_part = sch_acci_part.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
sch_acci_part

,학교급,사고부위,사고수
0,유치원,머리(두부),16983
1,유치원,손,5440
2,유치원,치아(구강),5026
3,유치원,발,4502
4,유치원,팔,3313
5,유치원,복합부위,3164
6,유치원,기타,2259
7,유치원,다리,1064
8,유치원,흉복부,303
9,초등학교,손,56248


In [46]:
# 각 연도별, 학교급별로 사고수가 가장 높은 5개의 사고형태 선택
top5_sch_acci_type = sch_acci_type.groupby(['연도', '학교급']).apply(lambda x: x.nlargest(5, '사고수')).reset_index(drop=True)
top5_sch_acci_type

C:\Users\cabby\AppData\Local\Temp\ipykernel_30920\3879063812.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\cabby\AppData\Local\Temp\ipykernel_30920\3879063812.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,연도,학교급,사고형태,사고수
0,2019,유치원,물리적힘 노출,3703
1,2019,유치원,낙상-넘어짐,2005
2,2019,유치원,기타,1229
3,2019,유치원,사람과의 충돌,1036
4,2019,유치원,낙상-미끄러짐,444
...,...,...,...,...
145,2023,기타학교,낙상,196
146,2023,기타학교,물리적힘 노출,175
147,2023,기타학교,염좌·삐임 등 신체 충격,122
148,2023,기타학교,사람과의 충돌,61


In [47]:
# 그래프 출력 함수 실행
create_chart(top5_sch_acci_type[top5_sch_acci_type['학교급']=='유치원'], '사고형태', '사고수', '연도', '유치원')

##### (2) 사고 당시 활동

In [70]:
sch_df['사고당시활동'].unique()

array(['기타운동', '보행/주행', '구기운동', '장난/놀이', '식사/수면/휴식', '기타', '실험실습', '공부'],
      dtype=object)

In [48]:
# 연도, 학교급, 사고당시활동별 사고수 계산
sch_acci_act = sch_df.groupby(['연도', '학교급', '사고당시활동']).size().reset_index(name='사고수')

# 연도 type str로 변경
sch_acci_act['연도'] = sch_acci_act['연도'].astype(str)

# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_acci_act['학교급'] = pd.Categorical(sch_acci_act['학교급'], categories=custom_order, ordered=True)
sch_acci_act = sch_acci_act.sort_values(['연도', '학교급']).reset_index(drop=True)

sch_acci_act

,연도,학교급,사고당시활동,사고수
0,2019,유치원,공부,565
1,2019,유치원,구기운동,179
2,2019,유치원,기타,1794
3,2019,유치원,기타운동,865
4,2019,유치원,보행/주행,1486
...,...,...,...,...
235,2023,기타학교,기타운동,62
236,2023,기타학교,보행/주행,120
237,2023,기타학교,식사/수면/휴식,27
238,2023,기타학교,실험실습,16


In [49]:
# 각 연도별, 학교급별로 사고수가 가장 높은 5개의 사고당시활동 선택
top5_sch_acci_act = sch_acci_act.groupby(['연도', '학교급']).apply(lambda x: x.nlargest(5, '사고수')).reset_index(drop=True)
top5_sch_acci_act

C:\Users\cabby\AppData\Local\Temp\ipykernel_30920\3903507569.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\cabby\AppData\Local\Temp\ipykernel_30920\3903507569.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,연도,학교급,사고당시활동,사고수
0,2019,유치원,장난/놀이,3043
1,2019,유치원,기타,1794
2,2019,유치원,보행/주행,1486
3,2019,유치원,기타운동,865
4,2019,유치원,식사/수면/휴식,739
...,...,...,...,...
145,2023,기타학교,구기운동,262
146,2023,기타학교,보행/주행,120
147,2023,기타학교,기타운동,62
148,2023,기타학교,기타,39


In [50]:
# 그래프 출력 함수 실행
create_chart(top5_sch_acci_act[top5_sch_acci_act['학교급']=='유치원'], '사고당시활동', '사고수', '연도', '유치원')

##### (3) 사고 매개물

In [71]:
sch_df['사고매개물'].unique()

array(['건물(문/창문/바닥/벽 등)', '기타', '자연(사람/동물/식물 등)', '가구(책상/의자/책장/탁자/침대 등)',
       '운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등)', '날카로운 물건(칼/가위/송곳 등)',
       '기계 도구류(기계선반, 재봉틀기계 등)', '열(불/뜨거운 물 등)', '운송용구(차/자전거/선박/항공기 등)'],
      dtype=object)

In [73]:
sch_acci_mdm = sch_df.copy()
sch_acci_mdm['사고매개물'] = sch_acci_mdm['사고매개물'].str.replace(r'\([^)]*\)', '', regex=True).str.strip()
# 연도, 학교급, 사고매개물별 사고수 계산
sch_acci_mdm = sch_acci_mdm.groupby(['연도', '학교급', '사고매개물']).size().reset_index(name='사고수')
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_acci_mdm['학교급'] = pd.Categorical(sch_acci_mdm['학교급'], categories=custom_order, ordered=True)
sch_acci_mdm = sch_acci_mdm.sort_values(['연도', '학교급']).reset_index(drop=True)
sch_acci_mdm

,연도,학교급,사고매개물,사고수
0,2019,유치원,가구,1842
1,2019,유치원,건물,1534
2,2019,유치원,기계 도구류,125
3,2019,유치원,기타,2182
4,2019,유치원,날카로운 물건,186
...,...,...,...,...
263,2023,기타학교,날카로운 물건,5
264,2023,기타학교,열,6
265,2023,기타학교,운동용 장비/기구,145
266,2023,기타학교,운송용구,6


In [51]:
# 연도, 학교급, 사고매개물별 사고수 계산
sch_acci_mdm = sch_df.groupby(['연도', '학교급', '사고매개물']).size().reset_index(name='사고수')

# 연도 type str로 변경
sch_acci_mdm['연도'] = sch_acci_mdm['연도'].astype(str)

# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_acci_mdm['학교급'] = pd.Categorical(sch_acci_mdm['학교급'], categories=custom_order, ordered=True)
sch_acci_mdm = sch_acci_mdm.sort_values(['연도', '학교급']).reset_index(drop=True)

# 그래프 해석 편의를 위해 괄호로 둘러싸인 부분 제거
sch_acci_mdm['사고매개물'] = sch_acci_mdm['사고매개물'].str.replace(r'\([^)]*\)', '', regex=True).str.strip()

sch_acci_mdm

,연도,학교급,사고매개물,사고수
0,2019,유치원,가구,1842
1,2019,유치원,건물,1534
2,2019,유치원,기계 도구류,125
3,2019,유치원,기타,2182
4,2019,유치원,날카로운 물건,186
...,...,...,...,...
263,2023,기타학교,날카로운 물건,5
264,2023,기타학교,열,6
265,2023,기타학교,운동용 장비/기구,145
266,2023,기타학교,운송용구,6


In [52]:
# 각 연도별, 학교급별로 사고수가 가장 높은 5개의 사고당시활동 선택
top5_sch_acci_mdm = sch_acci_mdm.groupby(['연도', '학교급']).apply(lambda x: x.nlargest(5, '사고수')).reset_index(drop=True)
top5_sch_acci_mdm

C:\Users\cabby\AppData\Local\Temp\ipykernel_30920\8947974.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\cabby\AppData\Local\Temp\ipykernel_30920\8947974.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,연도,학교급,사고매개물,사고수
0,2019,유치원,기타,2182
1,2019,유치원,가구,1842
2,2019,유치원,자연,1749
3,2019,유치원,건물,1534
4,2019,유치원,운동용 장비/기구,963
...,...,...,...,...
145,2023,기타학교,건물,168
146,2023,기타학교,운동용 장비/기구,145
147,2023,기타학교,기타,135
148,2023,기타학교,자연,97


In [53]:
# 그래프 출력 함수 실행
create_chart(top5_sch_acci_mdm[top5_sch_acci_mdm['학교급']=='유치원'], '사고매개물', '사고수', '연도', '유치원')

### 1-4. 사고 내용 분석

- 장소별 사고 현황
    - 장소별 사고 발생 건수 (누적? 연도별?)
    - 장소별 하루 평균 사고 발생 건수 : 해당 장소에서 하루에 얼마나 사고가 발생하는지?
    - 장소별 사고 부위, 사고 형태
- 시간별 사고 현황
    - 계절별 사고 발생 현황 : 계절별로 어떤 사고가 많이 발생하는지 (계절별 예방 대책 제시 가능)
        - 계절별 사고 건수
        - 계절별 사고 부위
        - 계절별 사고당시활동
    - 사고 시간별 사고 현황 (수업시간, 쉬는시간, 식사시간 등)
        - 사고 시간별 사고 건수
            - 사고 발생 시각별 건수(오전, 점심, 오후 정도의 비교가 의미 있지 않을지? <br>
            : 사고 시간별 건수 분석은 그냥 학교에서의 전체 시간이 수업시간, 쉬는 시간으로 뭉뚱그려진 게 좀 아쉬워서 오전 오후의 비교도 하면 어떨지 생각해봄, 애들이 밥먹고 하교 시간이 다가오면 좀 신나서 더 위험하다든가.. 그런 추측)
        - 사고 시간별 하루 평균 사고 발생 건수 : 해당 시간 대에 하루에 얼마나 사고가 발생하는지?
        - 사고 시간별 사고 형태, 사고 당시 활동, 사고 매개물 <br>
        : 사고 시간별로 어떤 사고가 많이 발생하는지? 등하교 시간에 자주 발생하는 사고 내용, 점심시간에 자주 발생하는 사고 내용 등 파악


In [11]:
df_2023.사고장소.unique()

array(['운동장', '통로', '부속시설', '교실', '교외'], dtype=object)

In [12]:
df_2023.사고시간.unique()

array(['식사시간', '체육활동', '수업시간', '학교행사', '특별활동', '쉬는시간', '등하교', '기타'],
      dtype=object)

In [14]:
df_2023.사고발생시각.unique()

array(['12:40', '11:20', '11:50', '13:20', '10:00', '13:45', '14:30',
       '11:00', '18:00', '13:50', '13:10', '15:00', '11:25', '10:40',
       '14:20', '09:55', '09:30', '12:10', '15:02', '15:40', '11:40',
       '13:40', '08:20', '14:50', '08:25', '14:00', '09:00', '10:10',
       '10:30', '15:20', '08:35', '09:26', '13:30', '08:30', '10:20',
       '09:20', '13:00', '17:10', '11:42', '16:00', '13:15', '12:15',
       '11:30', '12:25', '12:35', '15:30', '14:10', '12:00', '09:45',
       '12:50', '21:10', '10:15', '10:50', '13:05', '16:30', '12:30',
       '13:59', '16:15', '12:05', '08:40', '10:35', '15:50', '08:00',
       '11:55', '10:24', '10:38', '08:45', '14:15', '12:20', '15:32',
       '11:23', '12:45', '09:35', '09:50', '11:48', '14:53', '15:05',
       '11:35', '09:40', '17:00', '11:45', '16:10', '15:10', '11:10',
       '18:30', '13:25', '13:55', '09:10', '22:55', '10:55', '14:40',
       '13:08', '11:05', '14:55', '09:09', '15:45', '15:15', '12:32',
       '11:15', '08: